In [ ]:
playlist_urls = []
for i in range(41):
    playlist_urls.append(\
        f'https://sfs-msc-pub-lq-05.navigator.dot.ga.gov/rtplive/GDOT-CCTV-0'
        f'{500 + i}/playlist.m3u8')
playlist_urls

In [ ]:
import urllib.request
import requests
import re
import urllib
from pathlib import Path
import subprocess

for url in playlist_urls:
    base_url = url[:url.rfind('/') + 1]
    playlist = requests.get(url).text
    file_name = re.findall(r'chunklist_w.*?\.m3u8', playlist)
    if (len(file_name) == 0):
        continue
    file_name = file_name[-1]
    chunklist_url = base_url + file_name
    chunklist = requests.get(chunklist_url).text
    media_files = re.findall(r'media_w.*?\.ts', chunklist)
    Path(f'rawmedia/{base_url[-3:]}/').mkdir(exist_ok=True, parents=True)
    Path(f'media/{base_url[-3:]}/').mkdir(exist_ok=True, parents=True)
    for media_file in media_files:
        urllib.request.urlretrieve(base_url + media_file, f'rawmedia/{base_url[-3:]}/{media_file}')
        subprocess.run(['ffmpeg', 
                        '-i', 
                        f'rawmedia/{base_url[-3:]}/{media_file}', 
                        f'media/{base_url[-3:]}/{media_file[:-3]}.mp4'])
